In [1]:
import numpy as np
from tqdm import tqdm
from PIL import Image
import json
import os
import copy
import re

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

### 데이터셋 생성(이미지)

In [2]:
class Scalp_Health_Dataset(Dataset) :
    def __init__(self, image_path_list, vals_list) : # 용량을 고려해 이미지는 경로만 받는걸로
        self.image_path_list = image_path_list
        self.vals_list = vals_list
    def __len__(self) : 
        return len(self.image_path_list)

    def __getitem__(self, index) : # 한 개의 데이터 가져오는 함수
        # 224 X 224로 전처리
        to_tensor = transforms.ToTensor()
        img = to_tensor(Image.open(self.image_path_list[index]).convert('RGB'))
        img.resize_(3, 224, 224)
        img = torch.divide(img, 255.0) # 텐서로 변경 후 이미지 리사이징하고 각 채널을 0~1 사이의 값으로 만들어버림
        
        label = self.vals_list[index]
        
        return img, label


### 데이터셋 생성(두피 상태별 정도)

#### 데이터 분석 결과, 하나의 두피 이미지가 여러 증상을 가지고 있는 경우도 있음을 확인했다. 
#### 그래서 양호 0.2, 비듬 0.5...등으로 데이터를 만들려고 한다.  

In [3]:
# 이미지 데이터셋의 라벨에 해당하는 val1~val6이 여기서 입력값임
class Scalp_classifier_Dataset(Dataset) :
    def __init__(self, vals_list, severity_per_class_list) : # state_list : val1~val6 label_list : 증상별 중증도가 기록(예 : [0.33, 0.00, 1.00,...0.66])
        self.vals_list = vals_list
        self.severity_per_class_list = severity_per_class_list
    def __len__(self) : 
        return len(self.vals_list)

    def __getitem__(self, index) : # 한 개의 데이터 가져오는 함수
        
        state = self.vals_list[index] # val1~val6
        label = self.severity_per_class_list[index]
        
        return state, label

### 최종 데이터셋 생성

In [4]:
def make_dataset(dataset_path, category) : # root_path + '/Train'이나 root_path + '/Label'을 받음
    
    
    image_group_folder_path = dataset_path + '/Image'
    label_group_folder_path = dataset_path + '/Label'
    
    ori_label_folder_list = os.listdir(label_group_folder_path) # '[라벨]피지과다_3.중증' 등 폴더명 알기
    
    label_folder_list = []
    
    for i in range(len(ori_label_folder_list)) :
        if ori_label_folder_list[i] != '.DS_Store' : # '.DS_Store'가 생성되었을 수 있으니 폴더 목록에서 제외
            label_folder_list.append(ori_label_folder_list[i])
    
    image_path_list = []
    vals_list = []
    class_str_list = []
    
    desc_str = category + "_make_dataset"
    
    for i in tqdm(range(len(label_folder_list)), desc = desc_str) :
                  
        label_folder_path = label_group_folder_path + "/" + label_folder_list[i]
        
        # label_folder_list에서 '라벨'을 '원천'으로 만 바꿔도 image파일들이 들어있는 폴더명으로 만들 수 있다
        image_folder_path = image_group_folder_path + "/" + label_folder_list[i].replace('라벨', '원천')
        
        json_list = os.listdir(label_folder_path) # json파일 목록 담기

        for j in range(len(json_list)) : 
            json_file_path = label_folder_path + '/' + json_list[j]

            with open(json_file_path, "r", encoding="utf8") as f: 
                contents = f.read() # string 타입 
                json_content = json.loads(contents) # 딕셔너리로 저장

            image_file_name = json_content['image_file_name'] # 라벨 데이터에 이미지 파일의 이름이 들어있다
            
            image_file_path = image_folder_path + "/" + image_file_name

            # val1 ~ val6
            vals_true = []
            vals_true.append(int(json_content['value_1']))
            vals_true.append(int(json_content['value_2']))
            vals_true.append(int(json_content['value_3']))
            vals_true.append(int(json_content['value_4']))
            vals_true.append(int(json_content['value_5']))
            vals_true.append(int(json_content['value_6']))

            vals_true = torch.Tensor(vals_true).type(torch.float32)

            image_path_list.append(image_file_path)
            vals_list.append(vals_true/3.0)
            class_str_list.append(label_folder_list[i][4:]) # 이미지마다 할당된 클래스를 담음

    return image_path_list, vals_list, class_str_list
    # image_path_list : 파일 경로가 저장된 리스트
    # vals_list : val1 ~ val6이 들어있는 Tensor 리스트
    # class_str_list : "모낭사이홍반_0.양호" 등의 문자열이 저장된 리스트
    
    
    

In [5]:
# 하나의 모발 이미지가 여러 증상을 가진 경우가 있다.
# 하나의 이미지가 [A증상 중증, B증상 경증] 등 여러 증상에 대한 중증도를 나타내게끔 라벨 데이터를 만들어주는 기능도 한다
# 즉, 데이터 전처리
def make_unique_dataset(image_path_list, vals_list, class_str_list) :  
    unique_image_path_list = []
    unique_vals_list = []
    unique_severity_per_class_list = []
    
    state_str_list = ["미세각질", "피지과다", "모낭사이홍반", "모낭홍반농포", "비듬", "탈모", "양호"]
    
    for i in tqdm(range(len(image_path_list)), desc = "make unique dataset" ) : 
        file_name = image_path_list[i].split('/')[-1] # 이미지 파일 이름
        
        # 모낭사이홍반 등 증상이 적힌 문자열만 추출
        if class_str_list[i].find("중등도") != -1 :  
            class_str = class_str_list[i][:-6] 
        else :
            class_str = class_str_list[i][:-5]
        
        # 중증 정도
        severity_num = float(re.sub(r'[^0-9]', '', class_str_list[i]))
        severity = torch.Tensor([severity_num]).type(torch.float32)/3.0
        
        # 증상을 one-hot encoding형식으로 처리(1이 들어갈 자리에 1대신 중증도를 나타낸 숫자를 넣음)
        # 주의 : '양호'한 모발의 severity_per_class는 [0,0,0,0,0,0,1]이다
        if torch.eq(severity, 0.0) == True :
            class_str_index = state_str_list.index("양호")
        else : 
            class_str_index = state_str_list.index(class_str)
        
        severity_per_class = torch.zeros(len(state_str_list)).type(torch.float32)
        severity_per_class[class_str_index] = severity
        
        # 만들고 있던 unique list의 안에 같은 파일이름을 가진게 없는지 확인
        is_sameFilename_here = False
        for j in range(len(unique_image_path_list)) :
            if unique_image_path_list[j].split('/')[-1] == file_name : # 중복된 파일이 있으면
                # 클래스별 중증도만 통합
                unique_severity_per_class_list[j] = unique_severity_per_class_list[j] + severity_per_class
                is_sameFilename_here = True
                
        if is_sameFilename_here == False :
            unique_image_path_list.append(image_path_list[i])
            unique_vals_list.append(vals_list[i])
            unique_severity_per_class_list.append(severity_per_class)
    
    return unique_image_path_list, unique_vals_list, unique_severity_per_class_list

In [6]:
def get_dataset(root_path) : 
    
    Train_image_path_list, Train_vals_list, Train_class_str_list = make_dataset(root_path + '/Train', "Train")
    Train_image_path_list, Train_vals_list, Train_severity_per_class_list = make_unique_dataset(Train_image_path_list, Train_vals_list, Train_class_str_list)
    
    
    Test_image_path_list, Test_vals_list, Test_class_str_list = make_dataset(root_path + '/Test', "Test")
    Test_image_path_list, Test_vals_list, Test_severity_per_class_list = make_unique_dataset(Test_image_path_list, Test_vals_list, Test_class_str_list)
    
    
    
    Train_Scalp_Health_Dataset = Scalp_Health_Dataset(Train_image_path_list, Train_vals_list)
    Test_Scalp_Health_Dataset = Scalp_Health_Dataset(Test_image_path_list, Test_vals_list)
    
    Train_Scalp_classifier_Dataset = Scalp_classifier_Dataset(Train_vals_list, Train_severity_per_class_list)
    Test_Scalp_classifier_Dataset = Scalp_classifier_Dataset(Test_vals_list, Test_severity_per_class_list)
    
    return Train_Scalp_Health_Dataset, Test_Scalp_Health_Dataset, Train_Scalp_classifier_Dataset, Test_Scalp_classifier_Dataset
    
    

### 학습시키는 메소드

In [7]:
def train_model(model, criterion, optimizer, epochs, data_loader, device, desc_str) :
    # 학습 -> 성능 측정
    
    pred_loss = -1.0
    checkpoint_model = 0
    
    pbar = tqdm(range(epochs), desc = desc_str, mininterval=0.01)
    

    for epoch in pbar:  # loop over the dataset multiple times
            
        if desc_str == "training CNN" and (epoch == int(epochs * 0.5) or epoch == int(epochs * 0.75)) : # 31, 61번 째 에포크에서 학습률을 10배 줄임
            optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] / 10.0
            
        for inputs, labels in data_loader:
            
            # get the inputs
            inputs = inputs.to(device)
            labels = labels.to(device)
                
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            # print statistics
            pbar_str = "training CNN" + ", [loss = %.4f]" % loss.item()
            pbar.set_description(pbar_str)
            
            # 체크포인트
            # 배치 단위로 학습시킬 때마다 체크포인트 저장 여부 확인
            if pred_loss == -1.0 :
                pred_loss = loss
                checkpoint_model = copy.deepcopy(model)
            elif torch.lt(loss, pred_loss) == True : # loss를 더 줄였으면
                pred_loss = loss
                checkpoint_model = copy.deepcopy(model)
    
    # 체크포인트에 저장했던걸 최종 모델로
    model = copy.deepcopy(checkpoint_model)
    
    return model

### 구현

In [8]:
root_path = '/home/ubuntu/CUAI_2021/Advanced_Minkyu_Kim/Scalp_Health_Dataset'
# root_path = '/Users/minkyukim/Downloads/Scalp_Health_Dataset'
Train_Scalp_Health_Dataset, Test_Scalp_Health_Dataset, Train_Scalp_classifier_Dataset, Test_Scalp_classifier_Dataset = get_dataset(root_path)

make unique dataset: 100%|██████████| 2280/2280 [00:01<00:00, 1556.09it/s]


In [9]:
# 학습에 사용
BATCH_SIZE = 256

data_loader_Scalp_Health_Dataset = torch.utils.data.DataLoader(dataset=Train_Scalp_Health_Dataset, # 사용할 데이터셋
                                          batch_size=BATCH_SIZE, # 미니배치 크기
                                          shuffle=True, # 에포크마다 데이터셋 셔플할건가? 
                                          drop_last=True) # 마지막 배치가 BATCH_SIZE보다 작을 수 있다. 나머지가 항상 0일 수는 없지 않는가. 이 때 마지막 배치는 사용하지 않으려면 drop_last = True를, 사용할거면 drop_last = False를 입력한다

data_loader_Scalp_classifier_Dataset = torch.utils.data.DataLoader(dataset=Train_Scalp_classifier_Dataset, # 사용할 데이터셋
                                          batch_size=BATCH_SIZE, # 미니배치 크기
                                          shuffle=True, # 에포크마다 데이터셋 셔플할건가? 
                                          drop_last=True) 

In [10]:
# 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 에포크 설정
EPOCHS = 100

#### CNN 생성 : DenseNet_161에 classifier만 수정해서 학습시킨다

In [10]:
model_CNN = models.densenet161(pretrained = True, memory_efficient = True).to(device)

# ImageNet으로 학습시킨 CNN은 수정하지 못하게끔
for param in model_CNN.features.parameters():
    param.requires_grad = False

In [12]:
# 모발 분류를 위한 linear model 생성 후 교체
new_classifier = nn.Sequential(
            nn.Linear(in_features=2208, out_features=6, bias=True)
            , nn.Sigmoid() # 0~1사이 값으로 만들어버림
        ).to(device)

for m in new_classifier.modules():
    if isinstance(m, nn.Linear) :
        nn.init.kaiming_uniform_(m.weight)
        
model_CNN.classifier = new_classifier # 교체

NameError: name 'model_CNN' is not defined

In [15]:
# CNN 학습에 필요한 것들
optimizer_CNN = torch.optim.SGD(model_CNN.parameters(), lr=0.1, momentum = 0.9, weight_decay = 1e-4)
loss_CNN = nn.MSELoss() # 5종류의 출력값을 예측하는 선형회귀 모델이라 MSE사용

In [11]:
model_CNN = train_model(model_CNN, loss_CNN, optimizer_CNN, EPOCHS, data_loader_Scalp_Health_Dataset, device, "training CNN")

training, [loss = 0.0782]: 100%|██████████| 300/300 [19:39:14<00:00, 235.85s/it]  


In [12]:
# model_CNN(torch.unsqueeze(Train_Scalp_Health_Dataset[0][0], 0).to(device))

#### val1~val6을 입력값으로 받아 증상별 중증도를 출력하는 네트워크

In [11]:
model_Diagnoser = nn.Sequential(
            nn.Linear(in_features=6, out_features=64, bias=True),
            nn.LeakyReLU(),
            nn.Linear(in_features=64, out_features=128, bias=True),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=256, bias=True),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=128, bias=True),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=64, bias=True), 
            nn.LeakyReLU(),
            nn.Linear(in_features=64, out_features=32, bias=True), 
            nn.LeakyReLU(),
            nn.Linear(in_features=32, out_features=7, bias=False), 
            nn.Sigmoid()
        ).to(device)

In [12]:
loss_diagnoser = nn.MSELoss()
optimizer_diagnoser = torch.optim.Adam(scalp_state_diagnoser.parameters(), lr=0.001, weight_decay = 1e-4)

In [13]:
model_Diagnoser = train_model(model_Diagnoser, loss_diagnoser, optimizer_diagnoser, EPOCHS, data_loader_Scalp_classifier_Dataset, device, "training Diagnoser")

training CNN, [loss = 0.0178]: 100%|██████████| 100/100 [00:10<00:00,  9.67it/s]


#### 학습한 모델 저장하기

In [17]:
PATH = '/home/ubuntu/CUAI_2021/Advanced_Minkyu_Kim/Scalp_model_parameters/'

torch.save(model_CNN.state_dict(), PATH + 'model_CNN_parameter.pt')
torch.save(model_Diagnoser.state_dict(), PATH + 'model_Diagnoser_parameter.pt')